## **Eksempler: Multi-AI-agenter for hotellbestilling**

I dagens hektiske verden innebærer planlegging av en forretningsreise mer enn bare å bestille en flybillett og et hotellrom. Det krever en grad av koordinasjon og effektivitet som kan være utfordrende å oppnå. Det er her Multi-AI-agenter kommer inn i bildet og revolusjonerer måten vi håndterer våre reisebehov på.

Tenk deg å ha et team av intelligente agenter til din disposisjon, som samarbeider for å håndtere alle aspekter av reisen din med presisjon og enkelhet. Med vår avanserte AI-teknologi har vi utviklet spesialiserte agenter for bestillingstjenester og reiseplanlegging, som sikrer en sømløs og stressfri reiseopplevelse.

Dette er et grunnleggende scenario. Når vi planlegger en forretningsreise, må vi konsultere en forretningsreiseagent for å få informasjon om flybilletter, hoteller osv. Gjennom AI-agenter kan vi bygge agenter for bestillingstjenester og agenter for reiseplanlegging som samarbeider og forbedrer intelligensnivået.


# Initialiser Azure AI Agent Service og hent konfigurasjonsinformasjon fra **.env**

### **.env** 

Opprett en .env-fil 

**.env** inneholder tilkoblingsstrengen til Azure AI Agent Service, modellen som brukes av AOAI, og den tilsvarende Google API Search-tjenesten API, ENDPOINT, osv.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Navnet på din Azure AI Agent Service Model Deployment"

[**NOTE**] Du vil trenge en modell med 100,000 Rate Limit (Tokens per minutt) og Rate Limit på 600 (Forespørsler per minutt)

  Du kan få modellen i Azure AI Foundry - Model og Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Tilkoblingsstrengen til ditt Azure AI Agent Service-prosjekt"

  Du kan finne tilkoblingsstrengen til prosjektet i prosjektoversikten på AI ​​Foundry Portal-skjermen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

For å få Model Deployment Name og Project Connection String til Azure AI Agent Service, må du opprette Azure AI Agent Service. Det anbefales å bruke [denne malen](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) for å opprette den direkte （***Merk:*** Azure AI Agent Service er for øyeblikket satt til en begrenset region. Det anbefales at du refererer til [denne lenken](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) for å sette regionen)

Agenten må ha tilgang til SERPAPI. Det anbefales å registrere deg via [denne lenken](https://serpapi.com/searches). Etter registrering kan du få en unik API KEY og ENDPOINT.


# Logg inn på Azure

Du må nå logge inn på Azure. Åpne en terminal i VScode og kjør kommandoen `az login`.


# Oppsett

For å kjøre denne notatboken, må du installere følgende biblioteker. Her er en liste over de nødvendige bibliotekene og de tilsvarende pip install-kommandoene:

azure-identity: For autentisering mot Azure.  
requests: For å gjøre HTTP-forespørsler.  
semantic-kernel: For rammeverket semantic kernel (forutsatt at dette er et tilpasset eller spesifikt bibliotek, kan det hende du må installere det fra en spesifikk kilde eller et spesifikt repository).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Forklaring:
import asyncio: Dette importerer asyncio-modulen, som gir støtte for asynkron programmering i Python. Det lar deg skrive samtidig kode ved hjelp av async- og await-syntaksen.
from typing import Annotated: Dette importerer Annotated-typen fra typing-modulen. Annotated brukes til å legge til metadata til typehint, som kan være nyttig for ulike formål som validering, dokumentasjon eller verktøy.


In [ ]:
import asyncio,os
from typing import Annotated

# Forklaring:
Ved å bruke from dotenv import load_dotenv og load_dotenv() kan du enkelt håndtere konfigurasjonsinnstillinger og sensitiv informasjon (som API-nøkler og database-URLer) i en .env-fil. Dette holder dem adskilt fra kildekoden din og gjør applikasjonen din både sikrere og enklere å konfigurere.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Forklaring:

Importsetning: from azure.identity.aio import DefaultAzureCredential: Dette importerer klassen DefaultAzureCredential fra modulen azure.identity.aio. Delen aio i modulnavnet indikerer at den er designet for asynkrone operasjoner.

Formålet med DefaultAzureCredential: Klassen DefaultAzureCredential er en del av Azure SDK for Python. Den gir en standard måte å autentisere med Azure-tjenester på. Den forsøker å autentisere ved hjelp av flere metoder i en spesifikk rekkefølge, som miljøvariabler, administrert identitet og Azure CLI-legitimasjon.

Asynkrone operasjoner: Modulen aio indikerer at klassen DefaultAzureCredential støtter asynkrone operasjoner. Dette betyr at du kan bruke den med asyncio for å utføre ikke-blokkerende autentiseringsforespørsler.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Forklaring:
Importerer ulike moduler og klasser fra pakken semantic_kernel. Her er en oversikt over hver import:

AgentGroupChat fra semantic_kernel.agents: Denne klassen håndterer funksjonaliteter relatert til gruppechat for AI-agenter. AzureAIAgent og AzureAIAgentSettings fra semantic_kernel.agents.azure_ai

AzureAIAgent: Denne klassen brukes til å opprette og administrere AI-agenter som benytter Azure AI-tjenester.

AzureAIAgentSettings: Denne klassen brukes til å konfigurere innstillinger for AzureAIAgent. TerminationStrategy fra semantic_kernel.agents.strategies.termination.termination_strategy:

Denne klassen definerer strategier for å avslutte utførelsen av AI-agenter under visse betingelser. ChatMessageContent fra semantic_kernel.contents.chat_message_content:

Denne klassen brukes til å håndtere innholdet i chatmeldinger.
AuthorRole fra semantic_kernel.contents.utils.author_role:

Denne klassen definerer ulike roller for forfattere i konteksten av chatmeldinger.

kernel_function fra semantic_kernel.functions.kernel_function_decorator: Denne dekoratøren brukes til å definere kernel-funksjoner, som er funksjoner som kan utføres innenfor semantic kernel-rammeverket.
Disse importene setter opp de nødvendige komponentene for å opprette og administrere AI-agenter som kan samhandle i et gruppechatmiljø, muligens for oppgaver som å bestille hotell eller lignende aktiviteter.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Forklaring:
Neste importerer vi klassen CodeInterpreterTool fra modulen azure.ai.projects.models.

CodeInterpreterTool: Denne klassen er en del av Azure AI SDK og brukes til å tolke og utføre kode innenfor konteksten av AI-prosjekter. Den tilbyr funksjonaliteter for å kjøre kodebiter, analysere kode eller integrere kodeutførelse i AI-arbeidsflyter. 
Denne importen setter opp den nødvendige komponenten for å bruke CodeInterpreterTool i prosjektet ditt, noe som kan være nyttig for oppgaver som innebærer dynamisk tolkning og utførelse av kode.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Forklaring: 
ApprovalTerminationStrategy-klassen gir en spesifikk strategi for å avslutte en AI-agents operasjon. Agenten vil avslutte hvis den siste meldingen i interaksjonshistorikken inneholder ordet "lagret". Dette kan være nyttig i scenarier der agentens oppgave anses som fullført når den mottar bekreftelse på at noe er "lagret". Definer interaksjonsmetoden. Etter at reservasjonsplanen er lagret, kan den stoppes når den mottar signalet om at det er lagret.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Forklaring:

Linjen med kode initialiserer et AzureAIAgentSettings-objekt med standard- eller forhåndsdefinerte innstillinger ved å kalle create()-metoden. Dette innstillingsobjektet (ai_agent_settings) kan deretter brukes til å konfigurere og administrere en AzureAIAgent-instans.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Forklaring:
Ved å importere requests-biblioteket kan du enkelt gjøre HTTP-forespørsler og interagere med webtjenester i Python-koden din.


In [ ]:
import requests

# Forklaring:
Dette er en variabel som lagrer API-nøkkelen for å få tilgang til en SERP (Search Engine Results Page) API-tjeneste. En API-nøkkel er en unik identifikator som brukes til å autentisere forespørsler knyttet til kontoen din.

'GOOGLE_SEARCH_API_KEY': Dette er en plassholder-streng. Du må erstatte 'GOOGLE_SEARCH_API_KEY' med din faktiske SERP API-nøkkel.

Formål: Formålet med denne linjen er å lagre API-nøkkelen i en variabel slik at den kan brukes til å autentisere forespørsler til SERP API-tjenesten. API-nøkkelen er nødvendig for å få tilgang til tjenesten og utføre søk.

Hvordan få en SERP API-nøkkel: For å få en SERP API-nøkkel, følg disse generelle trinnene på https://serpapi.com (de nøyaktige trinnene kan variere avhengig av hvilken SERP API-tjeneste du bruker):

Velg en SERP API-tjeneste: Det finnes flere SERP API-tjenester tilgjengelig, som SerpAPI, Google Custom Search JSON API og andre. Velg den som passer best til dine behov.

Registrer deg for en konto:

Gå til nettsiden til den valgte SERP API-tjenesten https://www.serpapi.com og registrer deg for en konto. Du må kanskje oppgi litt grunnleggende informasjon og bekrefte e-postadressen din.

Opprett en API-nøkkel:

Etter registrering, logg inn på kontoen din og naviger til API-seksjonen eller dashbordet. Se etter et alternativ for å opprette eller generere en ny API-nøkkel.
Kopier API-nøkkelen:

Når API-nøkkelen er generert, kopier den. Denne nøkkelen vil bli brukt til å autentisere forespørslene dine til SERP API-tjenesten.
Erstatt plassholderen:

Erstatt plassholderen i .env-filen din


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Forklaring:
BookingPlugin-klassen tilbyr metoder for å bestille hoteller og flyreiser ved hjelp av Serpapi.com Google Search API. Den konstruerer nødvendige parametere, sender API-forespørsler og behandler svarene for å returnere relevant bestillingsinformasjon. API-nøkkelen (SERPAPI_SEARCH_API_KEY) og endepunktet (SERPAPI_SEARCH_ENDPOINT) brukes til å autentisere og sende forespørsler til Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Forklaring:
SavePlugin-klassen tilbyr en metode, saving_plan, for å lagre reiseplaner ved hjelp av Azure AI-tjenester. Den setter opp Azure-legitimasjon, oppretter en AI-agent, behandler brukerinnspill for å generere og lagre innholdet i reiseplanen, og håndterer fil-lagring og oppryddingsoperasjoner. Metoden returnerer "Saved" ved vellykket gjennomføring.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Forklaring:
Denne koden setter opp Azure AI-agenter for å håndtere bestilling av fly og hoteller, samt lagring av reiseplaner basert på brukerens input. Den bruker Azure-legitimasjon for å opprette og konfigurere agentene, behandler brukerens input gjennom en gruppechat, og sørger for riktig opprydding etter at oppgavene er fullført. Agentene bruker spesifikke plugins (BookingPlugin og SavePlugin) for å utføre sine respektive oppgaver.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
